In [ ]:
seed_value = 42
import os
os.environ["PYTHONHASHSEED"] = str(seed_value)

import random
random.seed(seed_value)


import numpy as np
np.random.seed(seed_value)


import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads = 1, inter_op_parallelism_threads = 1)
sess = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_excel(r"nlp_data.xlsx")

In [ ]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LeakyReLU, Flatten, SimpleRNN, LSTM, Bidirectional
import os
from bs4 import BeautifulSoup 
import tensorflow_addons as tfa
import re

In [ ]:
def cleanText(rawText):
    temp = BeautifulSoup(rawText, "html.parser")
    letters_only = re.sub("[!\"#\$%&\\\\'\(\)\*\+\,\-\./:;<\=>?@\[\]\^_`\{|\}~0-9]", "", temp.get_text())
    lower_case = letters_only.lower()
    return lower_case

In [ ]:
cleanX = []
for i in df.text:
    cleanX.append(cleanText(i))
    
print(cleanX)

['തേങ്ങ തേങ്ങാപ്പാൽ ഈസ്റ്റ് ഇവയൊന്നും ചേർത്തത് കാണിച്ചില്ലല്ലോ', 'thank you kaima rice doubt clear aayi eppol', 'വീണ ചേച്ചി ബ്രെഡ് ഒമ്ബ്ലെയ്ട് ', 'happy journey', 'when u come back mam', 'love your story veena thanks for sharing this made me feel like  you became a family member  as i keep watching whenever it comes up actually i watched your video a while ago  before thinking it was a christian testimony as i always watch changed lives of many this was real life i do watch your videos as it pops up almost everyday love to see you and your family often you have a very simple way of sharing and just love jan our brother simple young man role model and great example for many your respectappreciation  and love for jans parents was so sweet to hear praying that god will take care of you and  your family and its good to see you blessednever quit as you stayed strong in the most difficult time of your life really love to listen to the beautiful vision you had after avi was born and before se

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "https://www.manoramaonline.com/homestyle/spot-light/2019/10/31/lekshmi-nair-house-in-kochi-celebrity-home.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [ ]:
cleanX = np.array(cleanX)
print(cleanX.shape)

(4291,)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(cleanX, df.label, test_size = 0.25, random_state = 42)

In [ ]:
import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize

all_words = []
for sent in cleanX:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

unique_words = set(all_words)
print(len(unique_words))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
10465


In [ ]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(cleanX, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))
print(length_long_sentence)

185


In [ ]:
self_embedding_vocab_size = 10466
encoded_docs_se1 = [one_hot(d, self_embedding_vocab_size) for d in x_train]
max_length = 185
padded_docs_se1 = pad_sequences(encoded_docs_se1, maxlen = max_length, padding = "post")

In [ ]:
encoded_docs_se2 = [one_hot(d, self_embedding_vocab_size) for d in x_test]
padded_docs_se2 = pad_sequences(encoded_docs_se2, maxlen = max_length, padding = "post")

In [ ]:
print(padded_docs_se1.shape)

(4291, 185)


In [ ]:
model = Sequential()
e = Embedding(self_embedding_vocab_size, 20, input_length = max_length, trainable = False)
model.add(e)
model.add(Dense(20, input_dim = padded_docs_se1.shape[1], activation = "tanh"))
model.add(Dense(20, activation = "tanh"))
model.add(Flatten())
model.add(Dense(8, activation = "softmax"))
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()
model.fit(x = padded_docs_se1, y = df.label, epochs = 20, batch_size = 4)
_, accuracy = model.evaluate(padded_docs_se1, df.label, verbose = 0)
print('Accuracy: %f' % (accuracy*100))
y_pred = model.predict(padded_docs_se1)

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 185, 20)           209320    
_________________________________________________________________
dense_110 (Dense)            (None, 185, 20)           420       
_________________________________________________________________
dense_111 (Dense)            (None, 185, 20)           420       
_________________________________________________________________
flatten_26 (Flatten)         (None, 3700)              0         
_________________________________________________________________
dense_112 (Dense)            (None, 8)                 29608     
Total params: 239,768
Trainable params: 30,448
Non-trainable params: 209,320
_________________________________________________________________
Epoch 1/20
4291/4291 [==============================] - 2s 471us/step - loss: 1.6263 - accuracy: 0.4794
Epoc

In [ ]:
model = Sequential()
e = Embedding(self_embedding_vocab_size, 20, input_length = max_length, trainable = False)
model.add(e)
model.add(Dense(20, input_dim = padded_docs_se1.shape[1], activation = "relu"))
model.add(Dense(20, activation = "relu"))
model.add(Flatten())
model.add(Dense(8, activation = "softmax"))
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()
model.fit(x = padded_docs_se1, y = df.label, epochs = 20, batch_size = 4)
_, accuracy = model.evaluate(padded_docs_se1, df.label, verbose = 0)
print('Accuracy: %f' % (accuracy*100))
y_pred = model.predict(padded_docs_se1)

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_38 (Embedding)     (None, 185, 20)           209320    
_________________________________________________________________
dense_161 (Dense)            (None, 185, 20)           420       
_________________________________________________________________
dense_162 (Dense)            (None, 185, 20)           420       
_________________________________________________________________
flatten_37 (Flatten)         (None, 3700)              0         
_________________________________________________________________
dense_163 (Dense)            (None, 8)                 29608     
Total params: 239,768
Trainable params: 30,448
Non-trainable params: 209,320
_________________________________________________________________
Epoch 1/20
4291/4291 [==============================] - 2s 478us/step - loss: 1.6221 - accuracy: 0.4796
Epoc

In [ ]:
model = Sequential()
e = Embedding(self_embedding_vocab_size, 20, input_length = max_length, trainable = False)
model.add(e)
model.add(Dense(20, input_dim = padded_docs_se1.shape[1]))
model.add(LeakyReLU(alpha = 0.05)) 
model.add(Dense(20))
model.add(LeakyReLU(alpha = 0.05))
model.add(Flatten())
model.add(Dense(8, activation = "softmax"))
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()
model.fit(x = padded_docs_se1, y = df.label, epochs = 20, batch_size = 4)
_, accuracy = model.evaluate(padded_docs_se1, df.label, verbose = 0)
print('Accuracy: %f' % (accuracy*100))
y_pred = model.predict(padded_docs_se1)

Model: "sequential_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_47 (Embedding)     (None, 185, 20)           209320    
_________________________________________________________________
dense_197 (Dense)            (None, 185, 20)           420       
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 185, 20)           0         
_________________________________________________________________
dense_198 (Dense)            (None, 185, 20)           420       
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 185, 20)           0         
_________________________________________________________________
flatten_45 (Flatten)         (None, 3700)              0         
_________________________________________________________________
dense_199 (Dense)            (None, 8)               

In [ ]:
model = Sequential()
e = Embedding(self_embedding_vocab_size, 20, input_length = max_length, trainable = False)
model.add(e)

model.add(Bidirectional(LSTM(units = 64)))
model.add(LeakyReLU(alpha = 0.05))
model.add(Dense(64))
model.add(LeakyReLU(alpha = 0.05))

model.add(Dense(8, activation = "softmax"))
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()
model.fit(x = padded_docs_se1, y = y_train, epochs = 10, batch_size = 128)
_, accuracy = model.evaluate(padded_docs_se2, y_test, verbose = 0)
print('Accuracy: %f' % (accuracy*100))
y_pred = model.predict(padded_docs_se2)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 185, 20)           209320    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               43520     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 520       
Total params: 261,616
Trainable params: 52,296
Non-trainable params: 209,320
___________________________________________

In [ ]:
import sys
def prediction(y_pred):
    tempMax = -sys.maxsize
    labelValue = 0
    for i in range(0, 8):
        if(y_pred[i] > tempMax):
            tempMax = y_pred[i]
            labelValue = i
    
    return labelValue

predicted = []
for i in range(0, len(y_pred)):
    predicted.append(prediction(y_pred[i, :]))
    
print(accuracy_score(y_test, predicted))
print(matthews_corrcoef(y_test, predicted))

0.5097856477166822
0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
